# <center> *Big Data project* </center> 
### <center> *Attractive model* </center>
_<center>M2 Statistics & Econometrics, 2019-2020</center>_


In [0]:
import os
import zipfile
import random
import pickle
import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import expand_dims
from PIL import Image
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
!pip uninstall tensorflow -gpu -y
!pip install tensorflow-gpu==2.1.0
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras.layers import AveragePooling2D, MaxPool2D, Conv2D, MaxPooling2D, InputLayer 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback
from keras.utils import np_utils, to_categorical
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.models import load_model


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -g
     |████████████████████████████████| 421.8MB 40kB/s 
     |████████████████████████████████| 450kB 54.8MB/s 
     |████████████████████████████████| 3.9MB 55.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=cfb2661e51fb7580ddf3d07fbfd8cf0ffd8559f7b8387b338c15af5019004677
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorflow-estima

Using TensorFlow backend.


In [0]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with zipfile.ZipFile('/content/drive/My Drive/Big Data/Projet/celeba-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/img/')

In [0]:
path = '/content/img/img_align_celeba/img_align_celeba/'
samp = pickle.load(open('/content/drive/My Drive/Big Data/Projet/Listes sample/Attractive.pkl', 'rb'))

In [0]:
target = []
data = np.zeros((len(samp), 64, 64, 3)) #dimension of the images
count = 0
for file in os.listdir(path) :
  if file in samp :
    image = Image.open(path + file)
    image = image.resize((64, 64))
    data[count] = np.array(image)
    target.append(file)
    if count%1000 == 0 :
      print(count)
    count += 1
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [0]:
df_attr = pd.read_csv('/content/img/list_attr_celeba.csv')
df_attr.replace(to_replace = -1, value = 0, inplace = True)
attractive = df_attr[['image_id', 'Attractive']].set_index('image_id')
attractive = attractive.loc[target]
print(attractive.shape)
attractive.head()

(20000, 1)


,Attractive
image_id,
200257.jpg,0
163447.jpg,1
176844.jpg,1
179943.jpg,1
036919.jpg,1


### **Model**

In [0]:
target = to_categorical(attractive.Attractive)
print(target.shape)
print(data.shape)

(20000, 2)
(20000, 64, 64, 3)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state=0)
print("Train set dimension: ", X_train.shape, y_train.shape)
print("Test set dimension: ", X_test.shape, y_test.shape)

Train set dimension:  (16000, 64, 64, 3) (16000, 2)
Test set dimension:  (4000, 64, 64, 3) (4000, 2)


In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

model.compile(optimizers.Adam(lr = 0.001), loss = "categorical_crossentropy", 
              metrics = ["accuracy"])

In [0]:
model.fit(X_train, y_train, batch_size = 32, epochs = 10, 
          validation_data = (X_test, y_test))

Train on 16000 samples, validate on 4000 samples
Epoch 1/20
16000/16000 [==============================] - 13s 785us/sample - loss: 4.3920 - accuracy: 0.5854 - val_loss: 0.6336 - val_accuracy: 0.6417
Epoch 2/20
16000/16000 [==============================] - 6s 361us/sample - loss: 0.6213 - accuracy: 0.6607 - val_loss: 0.5973 - val_accuracy: 0.7000
Epoch 3/20
16000/16000 [==============================] - 6s 351us/sample - loss: 0.5533 - accuracy: 0.7269 - val_loss: 0.5942 - val_accuracy: 0.7015
Epoch 4/20
16000/16000 [==============================] - 6s 353us/sample - loss: 0.5325 - accuracy: 0.7501 - val_loss: 0.5900 - val_accuracy: 0.7125
Epoch 5/20
16000/16000 [==============================] - 6s 350us/sample - loss: 0.4710 - accuracy: 0.7783 - val_loss: 0.5803 - val_accuracy: 0.7258
Epoch 6/20
16000/16000 [==============================] - 6s 346us/sample - loss: 0.4393 - accuracy: 0.7954 - val_loss: 0.6355 - val_accuracy: 0.7115
Epoch 7/20
16000/16000 [==========================